Note you'll want to do this in a python 3 environment otherwise will run into pickle issues. 

Will also need the survey files I have pkl in galaxy_data one folder up from here ~ 10 Gb of data

The scripts to import from are at https://github.com/koconnor4/galaxy_catalogs

In [79]:
import numpy as np
import csv
from astropy.io import ascii
from astropy import cosmology as cosmo
from astropy.coordinates import SkyCoord,Angle  
from astropy.table import Table,vstack
import astropy.units as u
import pickle
import os
import matplotlib.pyplot as plt 

from candels_galaxy_surveys import galaxy_catalog,galaxy_survey

from update_galaxy_survey import update_survey

In [36]:
# my data directory is one up in a folder galaxy_data
# they are the galaxy surveys for the five fields
__current_dir__=os.path.abspath(os.getcwd())
__data_dir__=os.path.join(__current_dir__,"..","galaxy_data")
cos_survey =  pickle.load(open(os.path.join(__data_dir__,'cos.pkl'),'rb'))

cats = ['cos','uds','goodss','egs','goodsn']
gc = galaxy_catalog(cats)

Galaxy catalog uses those identifiers to go into data directory with the earlier pickled galaxy surveys for these available fields. For me the data directory is one folder up named galaxy_data. Five pkl survey objects one for each field. They are too large for github but can provide them for anyone interested in using this script.

They were created initially using the output_survey function in the script which is now irrelevant, not used for anything else.

The principal use of the galaxy_catalog is to find nearby galaxies to a given skycoord location. 

In [5]:
# giving a location at center of cosmos survey for demo
loc = SkyCoord('10:00:28.6','+02:12:21.0',unit = (u.hourangle,u.deg))

# the nearby function for catalog requires survey(s) to look through and a skycoord location
# it will default to searching within 5 arcseconds unless specified
# it will return a galaxy_survey of the galaxies inside angular region 
nearby_demo = gc.nearby('cos', loc, within=Angle(2.5/3600,u.deg))

Nayyeri2017; cos_candels; https://archive.stsci.edu/prepds/candels/ Nayyeri2017; cos_candels; https://archive.stsci.edu/prepds/candels/


The nearby function will accept list of multiple surveys to look through. For example could have used ['cos','uds'] as first argument and it would look through both of those. However I'd recommend using as few as possible as this is already a slow function; for single location takes a couple minutes to search a single survey. Use the location you want to look around to decide which survey is relevant.

The return from the nearby function is a galaxy_survey which is what is used primarily from this script. The galaxy_survey object is initialized using source, physpar, mass, photom. 

The sources are given in script, just links to website where data was taken from along with credit to author. The physpar, mass, photom are astropy tables I put together of the data from these authors. Don't need to worry about entering these as they are defaults which don't change in the data directory; now properties of all survey objects. 

Look now at the survey class.

In [44]:
# access a table (mass,phypsar, or photom); will stack the data for the different galaxies 
print('Nearby mass tables:')
print(nearby_demo.mass)

# so if you want to know how many galaxies in a survey can just ask for length of a table
print('cos has ',len(cos_survey.mass),' galaxies in survey')

# can ask for the locations of the galaxies in a survey
nearby_demo.gal_skycoords()

# the nearest function requires location, optional is conversion which defaults to the below
# default should be correct for all the surveys can omit
nearest_demo = nearby_demo.nearest(loc,conversion=3600.0/0.06)
print(nearest_demo)

Nearby mass tables:
  ID    Hmag (mag)  ... M_med_lin dex((solMass))  s_med_lin dex((solMass))
------- ----------- ... ------------------------- ------------------------
 2170.0      23.796 ...                      8.09                   0.0999
29757.0      27.172 ...                      7.84                    0.151
cos has  33950  galaxies in survey
Nayyeri2017; cos_candels; https://archive.stsci.edu/prepds/candels/ Nayyeri2017; cos_candels; https://archive.stsci.edu/prepds/candels/
[<candels_galaxy_surveys.galaxy_survey object at 0x12861bbe0>, 11.282535850709943, [11.282535850709943, 16.269793954432785]]


The nearest function for a survey returns list of three objects; 1: survey for nearest galaxy in terms of elliptical radii from the location 2: the elliptical radii of this nearest 3: the elliptical radii of all the other nearby.

the conversion is because we get seperation in degrees from skycoord sep of galaxy location in survey from given location that needs to go to arcseconds... and then to pixels which comes from underlying image. More complete description in the script function

The surveys also use source to get header values from table to access commonly requested parameters. Some currently available include redshift, coordinates, kron radius, elliptical parameters, sfr, age etc. To know the property names need to look in script. I should come up with some way to make this clearer. Also isn't obvious which table the parameter belongs to; need to come up with way for this to be indicated.

As an example lets look at a few parameters of the survey for nearest galaxy from our demo

In [77]:
nearest_survey = nearest_demo[0] # survey of nearest galaxy from our demo

age = nearest_survey.age_2a_tau # the name in table header needed to access
print(age)
print(nearest_survey.physpar[age])

kron = nearest_survey.kron_radius_names 
print(kron)
print(nearest_survey.photom[kron])

sfr = nearest_survey.sfr_2a_tau
print(sfr)
print(nearest_survey.physpar[sfr])

age_2a_tau (dex(yr)) 
8.5
KRON_RADIUS (pixel) 
4.320
SFR_2a_tau (solMass/yr) 
0.195


Occasionally the survey class is updated so that other parameters we want to look at are quickly accessible. If you have an old survey from previous pickle that was from before the most recent update to the class it is easy to 'upgrade' the survey so it has the most recent additions. 

You'll see something like the error below for the old cos data survey where I ask for the age which was not yet implemented when that pickle was made. 

To update use update_survey function from the script

In [76]:
cos_survey.age_2a_tau # old survey doesn't yet have this property

AttributeError: 'galaxy_survey' object has no attribute 'age_2a_tau'

In [80]:
# your most up-to-date script from latest commit has new property 
# and you get a message attribute error galaxy_survey has no attribute '' with old survey 
# try this update survey function 

update_survey(cos_survey)

Nayyeri2017; cos_candels; https://archive.stsci.edu/prepds/candels/ Nayyeri2017; cos_candels; https://archive.stsci.edu/prepds/candels/
